# Sales Forecast Demo

This notebook demonstrates:
- Querying recent data from BigQuery
- Calling the deployed Vertex AI Endpoint for online prediction



In [ ]:
import os
from google.cloud import bigquery, aiplatform
import pandas as pd

PROJECT_ID = os.environ.get("PROJECT_ID", "my-forecast-project-18870")
REGION = os.environ.get("REGION", "us-central1")
DATASET = os.environ.get("DATASET", "gk2_takeaway_sales")
FINAL_TABLE = os.environ.get("FINAL_TABLE", "daily_sales_features")

client = bigquery.Client(project=PROJECT_ID)

df = client.query(f"""
SELECT * FROM `{PROJECT_ID}.{DATASET}.{FINAL_TABLE}`
ORDER BY Date DESC
LIMIT 30
""").to_dataframe()
df.tail(3)


In [ ]:
# Call the deployed endpoint (set ENDPOINT_ID env var beforehand)
ENDPOINT_ID = os.environ.get("ENDPOINT_ID")
assert ENDPOINT_ID, "Set ENDPOINT_ID in environment to your deployed endpoint id"

aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint_name = ENDPOINT_ID if ENDPOINT_ID.startswith("projects/") else f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
endpoint = aiplatform.Endpoint(endpoint_name)

# Construct a single instance (dummy example, replace features accordingly)
instance = {"feature1": 1.0, "feature2": 2.0}

pred = endpoint.predict([instance])
pred.predictions
